# LLM-Knowledge

This is a handbook with all necessary knowledge about chatgpt and other LLMs for GeneRXN. 

## Chatgpt related pratical tips

### 1. Token limits

#### 1.1 What is a token for gpt and how do we count in real use

**What is a token** : Token is a concept in natural language which we use to break down a sentence into meta units, including words, punctuation marks... Depending on the tokenizing techniques used, a word can also be further broken down into prefix and postfix etc. According to Openai the approximation for tokens is: 
* 1 token ~= 4 chars in English

* 1 token ~= ¾ words

* 100 tokens ~= 75 words

Or 

* 1-2 sentence ~= 30 tokens

* 1 paragraph ~= 100 tokens

* 1,500 words ~= 2048 tokens 

#### 1.2 Token limitaions 

* **The limitation is shared between prompt and completion**. Prompt is your input and the completion is the answer from chatgpt. If you have a token limitation up to 4096 and your prompt is 4000 tokens, your completion can be 97 tokens at most. 

* Token limitations for different models

|     Model     | Maximum Tokens |
|:-------------:|:--------------:|
| gpt-3.5-turbo |  4,096 tokens  |
|     gpt-4     |  8,192 tokens  |
|   gpt-4-32k   |  32,768 tokens |

#### 1.3 Token pricing

* This is obtained from [Openai website](https://openai.com/pricing)

|     Model     |       Prompt       |     Completion     |
|:-------------:|:------------------:|:------------------:|
| gpt-3.5-turbo | $0.002 / 1K tokens | $0.002 / 1K tokens |
|     gpt-4     |  $0.03 / 1K tokens |  $0.06 / 1K tokens |
|   gpt-4-32k   |  $0.06 / 1K tokens |  $0.12 / 1K tokens |


### 2. Memory retenant (chat history preservation)

Think in this way, a model is basically a huge chunk of math calculations, therefore **there is no memory for the model**. Chatgpt and other chat bots use certain techniques to make chatgpt feel like it remember the previous chat history. One way to achieve that is by **feeding the previous messages to GPT**. To be more detailed, every single time we send a request to GPT, we have to firstly collect the chat log, attach the current prompt to it and then feed it to the model (Which is indeed inefficient when the conversation goes on and on).

An example is shown below, which is from the repo [here](https://github.com/atomic14/command_line_chatgpt/blob/main/main.py).
**The general logic is feeding the model the general context first, then every single time the model is fed by the combination of the chat log and current prompt.**

In [ ]:
def get_response(instructions, previous_questions_and_answers, new_question):
    """Get a response from ChatCompletion

    Args:
        instructions: The instructions for the chat bot - this determines how it will behave
        previous_questions_and_answers: Chat history
        new_question: The new question to ask the bot

    Returns:
        The response text
    """
    messages = [
        { "role": "system", "content": instructions },
    ]
    # add the previous questions and answers
    for question, answer in previous_questions_and_answers[-MAX_CONTEXT_QUESTIONS:]:
        messages.append({ "role": "user", "content": question })
        messages.append({ "role": "assistant", "content": answer })
    # add the new question
    messages.append({ "role": "user", "content": new_question })

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        top_p=1,
        frequency_penalty=FREQUENCY_PENALTY,
        presence_penalty=PRESENCE_PENALTY,
    )
return completion.choices[0].message.content

### 3. Work around token limitations